In [1]:
import pandas as pd
import numpy as np

#FOR VISUALIZATION:
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import emoji
import string
from os import path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker

In [2]:
from collections import Counter

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
df = pd.read_csv('D:/Semester 3/Temu Kembali Informasi/spotify 300 7 kata/dataset_300.csv')
df.columns = ['docID', 'text', ]
df

,docID,text
0,1,"Great music service, the audio is high quality..."
1,2,Please ignore previous negative rating. This a...
2,3,"This pop-up ""Get the best Spotify experience o..."
3,4,Dear Spotify why do I get songs that I didn't ...
4,5,The player controls sometimes disappear for no...
...,...,...
295,296,It provides best music quality than any other ...
296,297,I've been paying for my premium only for me no...
297,298,Let me stop autoplay when connecting to car bl...
298,299,"I'm sick of it, Spotify, please stop adding so..."


In [6]:
df.isnull().sum()

docID    0
text     0
dtype: int64

In [7]:
df = df.dropna(subset=['text']) 

In [8]:
def clean_text(text):
    text = re.sub(r'Ã[\x80-\xBF]+', ' ', text) 
    text = re.sub(r'[^a-zA-Z\s]', ' ', text) 
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text.lower()  # Convert all to lowercase

In [9]:
df.docID = pd.Series(["D" + str(ind) for ind in df.docID])
df

,docID,text
0,D1,"Great music service, the audio is high quality..."
1,D2,Please ignore previous negative rating. This a...
2,D3,"This pop-up ""Get the best Spotify experience o..."
3,D4,Dear Spotify why do I get songs that I didn't ...
4,D5,The player controls sometimes disappear for no...
...,...,...
295,D296,It provides best music quality than any other ...
296,D297,I've been paying for my premium only for me no...
297,D298,Let me stop autoplay when connecting to car bl...
298,D299,"I'm sick of it, Spotify, please stop adding so..."


In [10]:
df

,docID,text
0,D1,"Great music service, the audio is high quality..."
1,D2,Please ignore previous negative rating. This a...
2,D3,"This pop-up ""Get the best Spotify experience o..."
3,D4,Dear Spotify why do I get songs that I didn't ...
4,D5,The player controls sometimes disappear for no...
...,...,...
295,D296,It provides best music quality than any other ...
296,D297,I've been paying for my premium only for me no...
297,D298,Let me stop autoplay when connecting to car bl...
298,D299,"I'm sick of it, Spotify, please stop adding so..."


In [11]:
print(df.iloc[0]['text']) 

Great music service, the audio is high quality and the app is easy to use. Also very quick and friendly support.


In [12]:
print(df.iloc[283]['text']) 

Nice adio quality but shoul try to provide better quality for free


In [13]:
df['text'] = df['text'].apply(clean_text) 

In [14]:
df.to_csv('pre1.csv',index=False)

In [15]:
def tokenize_text(text):
    tokens = word_tokenize(text)  
    return tokens  

df['text'] = df['text'].apply(tokenize_text)

print(df.iloc[283]['text']) 

['nice', 'adio', 'quality', 'but', 'shoul', 'try', 'to', 'provide', 'better', 'quality', 'for', 'free']


In [16]:
df.to_csv('tokenisasi.csv', index=False)

In [17]:
# Menghapus stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))  # Get the set of English stopwords
    if custom_stopwords:
        stop_words.update(custom_stopwords)
    return [word for word in tokens if word.lower() not in stop_words]  # Filter out stopwords

In [18]:
custom_stopwords = {'app', 'music','play', 'spotify', 'song', 'songs', 'listen', 'playing','get', 'playlist'}
df['text'] = df['text'].apply(remove_stopwords)

In [19]:
df.head()

,docID,text
0,D1,"[great, service, audio, high, quality, easy, u..."
1,D2,"[please, ignore, previous, negative, rating, s..."
2,D3,"[pop, best, experience, android, annoying, ple..."
3,D4,"[dear, put, shuffle]"
4,D5,"[player, controls, sometimes, disappear, reaso..."


In [20]:
df.to_csv('stopword.csv', index=False)

In [21]:
import nltk
nltk.download('words')
from nltk.corpus import words

# Get the set of valid English words
valid_words = set(words.words())

# Function to remove words not in the dictionary
def remove_invalid_words(text):
    # Ensure text is a string
    if isinstance(text, list):
        text = ' '.join(text)
    return ' '.join([word for word in text.split() if word in valid_words])

# Apply the function to the cleaned text column
df['text'] = df['text'].apply(remove_invalid_words)

# Display the cleaned text for row 284
print("Cleaned text from row 284:")
print(df.iloc[283]['text'])


Cleaned text from row 284:
nice quality try provide better quality free


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dafa\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [22]:
print(df.iloc[283]['text']) 

nice quality try provide better quality free


In [23]:
df.head()

,docID,text
0,D1,great service audio high quality easy use also...
1,D2,please ignore previous negative rating super g...
2,D3,pop best experience android annoying please le...
3,D4,dear put shuffle
4,D5,player sometimes disappear reason restart issue


In [24]:

nltk.download('wordnet')
nltk.download('omw-1.4')


lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    if isinstance(text, list):
        text = ' '.join(text)
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

df['text'] = df['text'].apply(lemmatize_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dafa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dafa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [25]:
df.head()

,docID,text
0,D1,great service audio high quality easy use also...
1,D2,please ignore previous negative rating super g...
2,D3,pop best experience android annoying please le...
3,D4,dear put shuffle
4,D5,player sometimes disappear reason restart issue


In [26]:
df.head()

,docID,text
0,D1,great service audio high quality easy use also...
1,D2,please ignore previous negative rating super g...
2,D3,pop best experience android annoying please le...
3,D4,dear put shuffle
4,D5,player sometimes disappear reason restart issue


In [27]:
df.to_csv('stemming.csv', index=False)

In [28]:
df.head()

,docID,text
0,D1,great service audio high quality easy use also...
1,D2,please ignore previous negative rating super g...
2,D3,pop best experience android annoying please le...
3,D4,dear put shuffle
4,D5,player sometimes disappear reason restart issue


In [29]:
# Inisialisasi inverted index
inverted_index = {}

# Iterasi melalui dataframe
for index, row in df.iterrows():
    doc_id = row['docID']
    words = row['text']
    
    for word in words:
        if word not in inverted_index:
            inverted_index[word] = set()  # Buat set baru untuk kata ini
        inverted_index[word].add(doc_id)  # Tambahkan docID ke set

# Menampilkan hasil inverted index
for word, doc_ids in inverted_index.items():
    print(f"{word}: {', '.join(doc_ids)}")


g: D2, D133, D178, D108, D198, D290, D266, D219, D67, D222, D188, D210, D48, D21, D109, D264, D39, D81, D1, D268, D52, D86, D229, D214, D115, D17, D228, D54, D104, D138, D212, D14, D72, D218, D200, D149, D201, D146, D297, D260, D101, D122, D177, D69, D184, D209, D8, D232, D203, D49, D195, D244, D251, D237, D106, D270, D50, D20, D259, D230, D57, D6, D38, D9, D151, D78, D105, D23, D130, D265, D166, D116, D85, D97, D127, D287, D134, D225, D282, D3, D83, D156, D211, D91, D165, D161, D236, D240, D227, D269, D36, D117, D135, D168, D79, D126, D257, D285, D33, D94, D272, D128, D252, D7, D283, D171, D99, D59, D66, D73, D28, D249, D96, D216, D202, D206, D281, D176, D102, D142, D160, D169, D294, D143, D157, D58, D71, D41, D223, D68, D208, D261, D242, D186, D25, D215, D300, D27, D74, D129, D42, D233, D47, D125, D250, D173, D193, D278, D154, D288, D70, D256, D150, D289, D181, D132, D93, D280, D119, D262, D19, D141, D182, D60, D221, D140, D298, D114, D163, D137, D239, D170, D120, D254, D32, D90, D13

In [30]:
# Langkah 1: Mengonversi inverted index menjadi list of tuples
inverted_index_list = [(word, ', '.join(doc_ids)) for word, doc_ids in inverted_index.items()]

# Langkah 2: Membuat DataFrame
inverted_index_df = pd.DataFrame(inverted_index_list, columns=['Word', 'DocIDs'])

# Langkah 3: Menyimpan ke CSV
inverted_index_df.to_csv('inverted_index.csv', index=False)

In [31]:
# Menyimpan DataFrame ke CSV


In [32]:
# Langkah 2: Menghitung nilai TF-IDF


In [33]:
def clean_text_advanced(text):
    # Split into words
    words = text.split()
    # Remove single characters
    words = [word for word in words if len(word) > 2]
    # Join back into text
    return ' '.join(words)

# Apply the cleaning to the text
df['text'] = df['text'].apply(clean_text_advanced)

# Get all words from the cleaned texts
all_words = ' '.join(df['text']).split()
word_counts = Counter(all_words)

In [34]:
all_text = " ".join([" ".join(item) if isinstance(item, list) else item for item in df.text.values])
vocab = np.unique(word_tokenize(all_text))
vocab = [word for word in vocab if word not in stopwords.words('english')]
vocab

['ability',
 'able',
 'abroad',
 'absence',
 'absent',
 'absolutely',
 'acceptable',
 'access',
 'accessible',
 'accidentally',
 'account',
 'act',
 'acting',
 'action',
 'actually',
 'add',
 'added',
 'aesthetic',
 'aesthetically',
 'afford',
 'affordable',
 'ago',
 'alarm',
 'album',
 'allow',
 'almost',
 'along',
 'already',
 'also',
 'although',
 'always',
 'amazing',
 'amount',
 'android',
 'annoying',
 'another',
 'answer',
 'anyone',
 'anything',
 'anywhere',
 'apex',
 'appear',
 'apple',
 'application',
 'appropriate',
 'arbitrary',
 'army',
 'artist',
 'ask',
 'audio',
 'auto',
 'automatically',
 'automobile',
 'avail',
 'available',
 'avoid',
 'away',
 'awesome',
 'awhile',
 'baby',
 'back',
 'background',
 'bad',
 'bar',
 'bare',
 'barely',
 'based',
 'basic',
 'basically',
 'battery',
 'become',
 'becoming',
 'beginning',
 'behaviour',
 'besides',
 'best',
 'better',
 'big',
 'biggest',
 'birthday',
 'bit',
 'black',
 'blended',
 'bless',
 'blind',
 'blink',
 'bloat',
 'blo

In [35]:
def term_document_matrix(data, vocab=None, document_index='docID', text='text'):
    # Ensure each entry in the 'text' column is a single string
    data[text] = data[text].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
    
    # Initialize CountVectorizer with the provided vocabulary if given
    vectorizer = CountVectorizer(vocabulary=vocab)
    
    # Transform the text data into a term-document matrix
    X = vectorizer.fit_transform(data[text])
    
    # Convert the result to a DataFrame with words as rows and document IDs as columns
    vocab_index = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out(), columns=data[document_index])
    
    return vocab_index

In [36]:
similarity_index = term_document_matrix(df,vocab,'docID','text')

In [37]:
similarity_index.to_csv('termdoc.csv')

In [38]:
def tfidf_document_matrix(data, vocab=None, document_index='docID', text='text'):
    # Ensure text column contains strings
    data[text] = data[text].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
    
    # Initialize TfidfVectorizer with the provided vocabulary if given
    vectorizer = TfidfVectorizer(vocabulary=vocab, use_idf=True, smooth_idf=True)
    
    # Transform the text data into a TF-IDF matrix
    X = vectorizer.fit_transform(data[text])
    
    # Convert to DataFrame with words as rows and document IDs as columns
    tfidf_matrix = pd.DataFrame(
        X.T.toarray(),
        index=vectorizer.get_feature_names_out(),
        columns=data[document_index]
    )
    
    # Add document frequency and IDF as additional columns
    doc_freq = np.asarray(X.T.sum(axis=1)).ravel()
    tfidf_matrix['document_frequency'] = doc_freq
    tfidf_matrix['idf'] = vectorizer.idf_
    
    return tfidf_matrix

In [39]:
similarity_indez = tfidf_document_matrix(df,vocab,'docID','text')

In [40]:
similarity_indez.to_csv('tfidf.csv')

In [92]:
from sklearn.metrics.pairwise import cosine_similarity

In [96]:


# Step 1: Read the CSV file


# Step 2: Create TF-IDF matrix
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Step 3: Formulate the query
query = "audio"
query_vec = vectorizer.transform([query])

# Step 4: Calculate cosine similarity and rank documents
similarity_scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
ranked_indices = similarity_scores.argsort()[::-1]
ranked_docs = [(i, similarity_scores[i]) for i in ranked_indices]

# Step 5: Display the top ranked documents
top_n = 10  # Display top 10 results
for idx, score in ranked_docs[:top_n]:
    print(f"Document {idx}: Score {score}")
    print(df.iloc[idx]['text'])
    print()


Document 61: Score 0.5816425816945751
audio randomly control bar bottom screen unable pause stop skip audio without

Document 93: Score 0.4424076800157711
streaming trouble new update skipping audio

Document 0: Score 0.3303342709844792
great service audio high quality easy use also quick friendly support

Document 261: Score 0.2902534851672999
please fix broken every time galaxy update access control audio without control cost

Document 144: Score 0.0
used yet open press fix

Document 143: Score 0.0
dont know like fix

Document 145: Score 0.0
lite version think lite better like button skip close instead skipping undo option search also lot literally telling player also work background player like lite version getting rid one getting lite version

Document 94: Score 0.0
easy thank

Document 95: Score 0.0
amazing option completely fantasy

Document 96: Score 0.0
always popular stuff matter worst time little effort put made exclusively death metal dumb would trip dick racing suggest isla